## CS431/631 Data Intensive Distributed Analytics
### Winter 2020 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Yan Shi_
* **ID:** _20809455_

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [2]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [3]:
import random

num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.1415544


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

*The spark job here is to cut origin dataset into partitions, generate RDDs and do the parallel operation. In question 1, the origin dataset is the range from 0 to $100000000$, for each item in the dataset, we generate a random number $(x,y)$, where $x \in (0,1)$ and $y \in (0,1)$, since we only used $x^2$ and $y^2$ here, $x$ and $y$ can also be regard as negative, this means all these points are in the square with vertices $(-1,1)$, $(1,1)$, $(-1,-1)$, $(1,-1)$. Also, there is a inscribed circle with center $(0,0)$ and radius $1$ in this square, the function above is to select points which are in the inscribed circle from all the $100000000$ points in square. Then we estimate the area of circle by the following formula: $$\frac{area\ of\ circle}{area\ of\ square} = \frac{number\ of\ points\ in\ circle}{number\ of\ points\ in\  square}$$ then knowing that ${area\ of\ circle} = {\pi}*1^2 = {\pi}$, ${area\ of\ square} = 2^2 = 4$, we get the formula for estimating ${\pi}$: ${\pi} = 4*\frac{number\ of\ points\ in\ circle}{number\ of\ points\ in\  square}$, which is what we did in the above python code.*

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [4]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    # your solution to Question 2 here
    lines = sc.textFile("Shakespeare.txt")
    words = lines.flatMap(simple_tokenize).distinct()
    tokenCount = words.count()
    return tokenCount

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [14]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    import itertools
    # your solution to Question 3 here
    lines = sc.textFile("Shakespeare.txt")
    tokenPairs = lines.flatMap(lambda l: itertools.product(simple_tokenize(l), simple_tokenize(l)))
    tokenPairs = tokenPairs.filter(lambda tp: tp[0]!=tp[1]).distinct()
    tokenPairsCount = tokenPairs.count()
    return tokenPairsCount

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [19]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():

    # your solution to Question 4 here
    lines = sc.textFile("Shakespeare.txt")
    numLines = lines.count()
    tokens = lines.flatMap(lambda x: list(set(simple_tokenize(x))))
    tokenCount = tokens.countByValue()
    tokenCount = sc.parallelize(dict(tokenCount).items())
    tokenCount50 = tokenCount.takeOrdered(50, lambda x: -x[1])
    
    def three_items_pair(i):
        '''
        numlines: number of lines of the text; being used to calculate probability

        wordCount: list of tuples, first item is token, second item is number of occurence
        '''
        return (i[1]/numLines, i[1], i[0])
    
    tokenCount50 = sc.parallelize(tokenCount50).map(lambda x: three_items_pair(x)).collect()
    
    return tokenCount50

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [21]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # your solution to Question 5 here
    
    def calPMI(x, y, xy): 
        return log((xy/n) / ((x/n) * (y/n)), 10)
    
    def tokenpairs(l):
        pairsList = []
        l = simple_tokenize(l)
        for i in range(0, len(l)):
            for j in range(i+1, len(l)):
                if l[i] != l[j]:
                    pairsList.append(tuple(sorted([l[i], l[j]])))
        return list(set(pairsList))
            
    lines = sc.textFile("Shakespeare.txt")
    n = lines.count()
    tokens = lines.flatMap(lambda x: list(set(simple_tokenize(x))))
    tokensCount = sc.parallelize(tokens.countByValue().items())
    tokenPairs = lines.flatMap(tokenpairs)
    tPCount = sc.parallelize(tokenPairs.countByValue().items()).filter(lambda x: x[1]>=threshold)
    
    results = tPCount.map(lambda x: (x[0][0], (x[0], x[1]))).leftOuterJoin(tokensCount)
    results = results.map(lambda x: (x[1][0][0][1], (x[0], x[1]))).leftOuterJoin(tokensCount)
    results = results.map(lambda x: (x[1][0][1][0][0], calPMI(x[1][0][1][1], x[1][1], x[1][0][1][0][1]),\
                                     x[1][0][1][0][1], x[1][0][1][1], x[1][1]))
    
    return results.collect()

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [27]:
from simple_tokenize import simple_tokenize
from math import log
import itertools

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here
    
    def one_token_pair(line):
        l = list(set(simple_tokenize(line)))
        tpList = []
        for i in l:
            if i in tokenList:
                tpList.extend(list(itertools.product([i], l)))
                tpList.remove((i,i))
        return tpList
    
    def calPMI(x, y, xy): 
        return log((xy/n) / ((x/n) * (y/n)), 10)
    
    lines = sc.textFile("Shakespeare.txt")
    n = lines.count()
    tokens = lines.flatMap(lambda x: list(set(simple_tokenize(x))))
    tokenList = tokens.distinct().takeSample(withReplacement=False, num = samp_size)
    tokensCount = sc.parallelize(tokens.countByValue().items())
    oneTokenPairs = lines.flatMap(one_token_pair)
    pairsCount = sc.parallelize(oneTokenPairs.countByValue().items()).filter(lambda x: x[1]>=threshold)
    
    results = pairsCount.map(lambda x: (x[0][0], (x[0], x[1]))).leftOuterJoin(tokensCount)
    results = results.map(lambda x: (x[1][0][0][1], (x[0], x[1]))).leftOuterJoin(tokensCount)
    results = results.map(lambda x: (x[1][0][1][0][0], calPMI(x[1][0][1][1], x[1][1], x[1][0][1][0][1]),\
                                     x[1][0][1][0][1], x[1][0][1][1], x[1][1]))
    results = results.map(lambda x: (x[0][0],x)).groupByKey().map(lambda x: (x[0], list(x[1])))
    
    return results.collect()

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.